### コーパスの中身をkmeansでクラスタリング

In [1]:
from collections import defaultdict

from gensim import corpora
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
model = KeyedVectors.load('output/word2vec_ramen_model.model')

In [3]:
max_vocab = 30000
vocab = list(model.wv.key_to_index.keys())[:max_vocab]
vectors = [model.wv[word] for word in vocab]

In [4]:
# クラスター数はこちらで任意の値を定める
n_clusters = 6
kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42)
kmeans_model.fit(vectors)

KMeans(n_clusters=6, random_state=42)

In [5]:
cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, vocab):
    cluster_to_words[cluster_id].append(word)
    
for label, words in cluster_to_words.items():
    print(label, words[:20])

2 ['スープ', '醤油', '感じ', '濃厚', '旨味', '煮干し', '出汁', '味わい', '香り', '風味', '魚介', '良く', 'なく', 'バランス', '最後', '印象', '旨い', '見た目', 'ストレート', '豚骨']
1 ['ラーメン', '0円', 'こと', 'よう', 'これ', 'そう', 'つけ麺', 'ない', 'そば', 'もの', 'メニュー', '中華そば', '特製', 'らーめん', '今回', '味噌', 'それ', '好き', 'ところ', 'ここ']
5 ['チャーシュー', '美味しい', '良い', 'メンマ', 'いい', 'トッピング', 'つけ', 'ネギ', 'ワンタン', '食感', '野菜', '海苔', '好み', '具材', 'ご飯', '美味い', '美味しく', '味付け', '調理', 'シンプル']
0 ['0分', '店内', 'カウンター', '食券', '注文', '到着', '券売機', '購入', '待ち', '店主', '時間', '入店', '開店', '店員', '雰囲気', '着席', '先客', 'オーダー', 'お客さん', '丁寧']
3 ['さん', '0時', '訪問', 'こちら', '人気', '行列', '今日', 'ごちそうさま', '平日', 'ラーメン屋', '徒歩', '近く', '食べログ', '場所', 'ランチ', '営業', '名店', 'オープン', '以前', 'ラーメン店']
4 ['https', '.com', 'www', 'instagram', '0%', 'twitter', 'グルメ', 'tokyo', 'ブログ', 'youtube', 'http://', 'entry', '.jp', 'blog', 'u.', 'yo', 'ut', 'be', 'スタ', '詳しく']


In [6]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

In [7]:
# # 各クラスターの解釈を決定
change_dict_key(cluster_to_words, 1,  'ラーメンに関するワード')
change_dict_key(cluster_to_words, 2, 'スープに関するワード')
change_dict_key(cluster_to_words, 5, 'トッピングに関するワード')
change_dict_key(cluster_to_words, 0, '店内に関するワード')
change_dict_key(cluster_to_words, 3, '店舗に関するワード')
change_dict_key(cluster_to_words, 4, 'その他のワード')

In [8]:
df = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
df.to_csv('output/words_cluster.csv')
df.head(20)

,ラーメンに関するワード,スープに関するワード,トッピングに関するワード,店内に関するワード,店舗に関するワード,その他のワード
0,ラーメン,スープ,チャーシュー,0分,さん,https
1,0円,醤油,美味しい,店内,0時,.com
2,こと,感じ,良い,カウンター,訪問,www
3,よう,濃厚,メンマ,食券,こちら,instagram
4,これ,旨味,いい,注文,人気,0%
5,そう,煮干し,トッピング,到着,行列,twitter
6,つけ麺,出汁,つけ,券売機,今日,グルメ
7,ない,味わい,ネギ,購入,ごちそうさま,tokyo
8,そば,香り,ワンタン,待ち,平日,ブログ
9,もの,風味,食感,店主,ラーメン屋,youtube


### TF-IDFで文章における特徴的な単語を抽出

In [9]:
from gensim import corpora
from gensim import models

In [10]:
# 口コミからレコメンドに使用する情報を絞る
# 試行錯誤して決定するべし!!
ramen_words = cluster_to_words['ラーメンに関するワード']
soup_words = cluster_to_words['スープに関するワード']
topping_words = cluster_to_words['トッピングに関するワード']

ramen_words.extend(soup_words)
ramen_words.extend(topping_words)
key_words = ramen_words

In [11]:
with open('output/ramen_corpus.txt', 'r', encoding='utf-8') as f:
    trainings = []
    for i, data in enumerate(f):
        # 不要な文字を取り除いて単語を取り出す
        words = data.replace("'", '').replace('[', '').replace(']', '').replace(' ', '').replace('\n', '').split(",")
        trainings.append([i for i in words if i in key_words])

In [12]:
# 単語->id変換の辞書作成
dictionary = corpora.Dictionary(trainings)

In [13]:
# textsをcorpus化
corpus = list(map(dictionary.doc2bow, trainings))

In [14]:
# tfidf modelの生成
test_model = models.TfidfModel(corpus)

In [15]:
# corpusへのモデル適用
corpus_tfidf = test_model[corpus]

In [16]:
# id->単語へ変換
texts_tfidf = []    # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]], word[1]])
    texts_tfidf.append(text_tfidf)

In [17]:
from operator import itemgetter
import pickle

In [18]:
texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [28]:
# 結果をデータフレームに追加
store_df = pd.read_csv('output/ramen_store.csv')
review_df = pd.read_csv('output/ramen_review.csv')
df_ramen = pd.merge(store_df, review_df)

In [26]:
df_ramen = df_ramen.groupby(['ID', 'score'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
df_ramen['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20

df_ramen_texts_tfidf_sorted_top20 = df_ramen.iloc[:,[0, 3]].reset_index(drop=True)
df_ramen_texts_tfidf_sorted_top20.head()

,ID,texts_tfidf_sorted_top20
0,1,"[ワンタン, 昆布, 炭火, 海老, 特製, 火炙り, だれ, 炙り焼, 焼き豚, 藻塩, ..."
1,2,"[黒毛和牛, いりこ, 深谷, 火山, spf, spf豚, ワンタン, 手入れ, 鰹節, ..."
2,3,"[ワンタン, 手打, 中華そば, 三つ葉, 特製, 鶏胸肉, 生醤油, 紫蘇, うどん, 大..."
3,4,"[軍鶏, 替え玉, そば, まぜそば, しゃも, 卵黄, 醤油漬け, 素地, 燻製, 特製,..."
4,5,"[しじみ, 宍道湖, 島根県, シジミ, 琥珀, ワンタン, 雑色, バラ, コハク酸, 地..."


In [27]:
# 上の結果をダンプ
with open('output/df_ramen_texts_tfidf_sorted_top20', 'wb') as f:
    pickle.dump(df_ramen_texts_tfidf_sorted_top20, f)

In [29]:
df_ramen_texts_tfidf_sorted_top20.to_csv('output/tfidf_top20.csv')